CRISPR TXTL Experiment: comparing 1 or more different RNA guides across a range of concentrations


In [1]:
experiment_name = 'RMS-experiment-1-dcas9-a'

# set your reaction volume:

reaction_volume = 5e-6 # 2 microliters

# set your mastermix ratio

master_mix_ratio = 0.60 # 75% 

# enter concentrations for your starting source solutions

# enter your desired finnal concentrations

chi_concentration = 2.5e-6
nuclease_dna_concentration = 1e-9

experiment_polymerase_dna_concentration = [0]
experiment_reporter_dna_concentrations = [2.0e-9]
experiment_guideRNA_dna_concentration = [0, 1e-9]
experiment_sgrna_concentrations = [0.5e-6, 1e-6, 2e-6]

# create labels for each of your guides to test
targets = [
    'g1',
    'g2',     
    'nt',
]

# fixed concentrations
experiment_starting_conc = {
    'chi' : 30e-6,
    'reporter' : 12e-9,
    'nuclease' : 100e-9,
    'polymerase' : 9.8e-9,
    'no guide control' : 0,
    'no reporter control' : 0,
    'blank control' : 0,
    'sgrna' : 50e-6,
}

experiment_rna_guides = targets

# and finally set the number of replicates for each run

replicates = 3

# plate layout

dest_layout = (12,8) # 96 well

# dest_layout = (24,16) # 384 

src_layout = (24,16) # 384

# we assume you're using the standard source plate (15-65µL working range)
b_use_ldv = False

if b_use_ldv:
    transfer_range = (2.5e-6, 12e-6)
else:
    transfer_range = (15e-6, 65e-6)

In [2]:
# import python things here
import pandas as pd
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [3]:
class EchoManager():
    
    def __init__(self, src_layout, src_transfer_range, dest_layout, reaction_volume, 
                 master_mix_ratio=0.75, 
                 starting_concentrations={}):
        
        
        ##TODO refactor
        self.starting_concentration = starting_concentrations
        
        self.src_aliquots = {}
        self.aliquots = []
        self.plate_row_index = 0
        self.dst_plate_index = 0
        
        self.min_transfer_size = 2.5 * 1e-3 # captured, but not needed 
        self.reaction_volume = reaction_volume
        self.txtl_mix_volume = reaction_volume * master_mix_ratio
        self.available_input_volume = reaction_volume - self.txtl_mix_volume
        
        self.src_layout = src_layout
        self.dest_layout = dest_layout
        self.src_transfer_range = src_transfer_range
        
        self.final_source_df = None
        self.effective_transfer_volume = src_transfer_range[1] - src_transfer_range[0]
            
    def src_plating_index_to_str(self, index):
        row = floor(index/self.src_layout[0])
        col = index % self.src_layout[0]
        return '%s%02d' % (chr(int(row + 65)), col+1)

    def dst_plating_index_to_str(self, index):
        row = floor(index/self.dest_layout[0])
        col = index % self.dest_layout[0]
        return '%s%02d' % (chr(int(row + 65)), col+1)
    

    def add_rna_targets(self, targets):
        for t in targets:
            self.starting_concentration[t] = 24e-9

    def generate_source_plating(self):
        final_source_plating = []

        for k,v in self.src_aliquots.items():
            for a in v:        
                user_step = {
                    'reagent' : k,
                    'well' : a['well_str'],
                    'volume' : (a['active_volume_ul']),
                    'active_volume_ul' : (a['active_volume_ul']) / 1e-6,
                    'volume_plus' : a['active_volume_ul'] + self.src_transfer_range[0]            
                }

                user_step['volume_ul']  = user_step['volume_plus'] / 1e-6

                final_source_plating.append(user_step)

        self.final_source_df = pd.DataFrame(final_source_plating)
        
        return self.final_source_df.sort_values(by='well')

    
    def generate_transfers(self):
        
        transfer_df = pd.DataFrame(self.aliquots)
        self.transfer_df = transfer_df
        
        final_xfers = []
        for n,transfer in transfer_df.iterrows():

            for rgt in ['master', 'chi', 'h20', 'nuclease', 'rna', 'sgrna', 'reporter']:

                rgt_source_well = '%s_source' % rgt
                rgt_source_name = '%s_source_name' % rgt
                rgt_source_vol = '%s_ul' % rgt

                xfer = {

                    'condition' : transfer['name'],

                    'rgt' : rgt,
                    'source_name' : transfer[rgt_source_name],

                    'dst_volume' : transfer[rgt_source_vol] / 1e-9, # we output the csv in nL

                    'dst_plate_label' : 'dstPlate1',
                    'src_plate_label' : 'srcPlate1',

                    'src_well' : transfer[rgt_source_well],
                    'dst_well' : transfer['index_str'], 

                    'pk' : transfer['index']
                }
                
                if transfer[rgt_source_vol] > 0:
                    final_xfers.append(xfer) 
                
    
        RGT_RANK = {
            'master' : 0,
            'chi' : 1,
            'h20' : 2,
            'polymerase' : 3,
            'reporter' : 4,         
            'nuclease' : 5,
            'rna' : 6,
            'sgrna' : 7
        }    
        
        def rank_by_reagent(row):
            if row['rgt'] in RGT_RANK:
                return RGT_RANK[row['rgt']]
            return 999

        def plate_transfer_by_reagent(row):
            if row['rgt'] == 'master':
                return '384PP_AQ_CP'
            return '384PP_AQ_BP2'
        
        final_xfers_df = pd.DataFrame(final_xfers)
        final_xfers_df['rgt_rank'] = final_xfers_df.apply(rank_by_reagent, axis=1)
        final_xfers_df['Source Plate Type'] = final_xfers_df.apply(plate_transfer_by_reagent, axis=1)
        
        final_xfers_df = final_xfers_df.sort_values(by=['rgt_rank', 'pk'])        
        self.final_xfers_df = final_xfers_df
        
    def generate_pre_chi_transfer(self):
        prechi = self.final_xfers_df[self.final_xfers_df.rgt_rank < 4][['condition','src_plate_label', 'src_well', 'dst_volume', 'dst_plate_label', 'dst_well', 'Source Plate Type']]
        prechi.columns = ['Sample ID', 'Source Plate Name', 'Source Well', 'Transfer Volume', 'Destination Plate Name', 'Destination Well', 'Source Plate Type']        
        return prechi

    def generate_post_chi_transfer(self):
        postchi = self.final_xfers_df[self.final_xfers_df.rgt_rank >= 4][['condition', 'src_plate_label', 'src_well', 'dst_volume', 'dst_plate_label', 'dst_well', 'Source Plate Type']]
        postchi.columns = ['Sample ID', 'Source Plate Name', 'Source Well', 'Transfer Volume', 'Destination Plate Name', 'Destination Well', 'Source Plate Type']
        return postchi
        
    def create_aliquots_for_crispr_txtl(self, rna_guides, reporter_dna_concentrations, guideRNA_dna_concentration, sgrna_concentrations):
        for guide_name in rna_guides:
            for dna_conc in reporter_dna_concentrations:
                
                for rna_conc in guideRNA_dna_concentration:     
                    
                    real_sg_concs = sgrna_concentrations
                    if rna_conc > 0:
                        real_sg_concs = [0]                        
                
                    for sg_conc in real_sg_concs:
                    
                        nmol_reporter = dna_conc * reaction_volume
                        nmol_rna = rna_conc * reaction_volume
                        nmol_chi = chi_concentration * reaction_volume
                        nmol_nuclease = nuclease_dna_concentration * reaction_volume

                        
#                         if nmol_rna > 0:
#                             sg_conc = 0
                            
                        nmol_sgrna = sg_conc * reaction_volume

                        if self.starting_concentration['reporter'] > 0:
                            ul_reporter = nmol_reporter / self.starting_concentration['reporter']
                        else:
                            ul_reporter = 0

                        if self.starting_concentration[guide_name] > 0:
                            ul_rna = nmol_rna / self.starting_concentration[guide_name]
                        else:
                            ul_rna = 0

#                         if self.starting_concentration['sgrna'] > 0:
                        ul_sgrna = nmol_sgrna / self.starting_concentration['sgrna']
#                         else:
#                             ul_sgrna = 0
                            
                        ul_nuclease = nmol_nuclease / self.starting_concentration['nuclease']
                        ul_chi = nmol_chi / self.starting_concentration['chi'] 
                        
                        

                        ul_final_h20 = self.reaction_volume - self.txtl_mix_volume - (ul_reporter + ul_rna + ul_chi + ul_nuclease + ul_sgrna)        

                        for rep in range(replicates):

                            aliquot_name = '%s - %0.2fnM-rep x %0.2fnM guide plasmid x %0.2fnM guide sgRNA  (%d)' % (guide_name, dna_conc*1e9, rna_conc*1e9, sg_conc*1e9, rep)

                            aliquot = {
                                'guide': guide_name,
                                'name' : aliquot_name,
                                'reporter_ul' : ul_reporter,
                                'master_ul' : self.txtl_mix_volume,
                                'nuclease_ul': ul_nuclease,
                                'rna_ul' : ul_rna,
                                'sgrna_ul' : ul_sgrna,
                                'chi_ul' : ul_chi,
                                'h20_ul' : ul_final_h20,
                                'index' : self.dst_plate_index,
                                'index_str' : self.dst_plating_index_to_str(self.dst_plate_index)
                            }


                            # increment the plate index to the next loction
                            self.dst_plate_index += 1

                            for rgt in ['master', 'chi', 'h20', 'nuclease', 'rna', 'sgrna', 'reporter']:

                                # what is the reagent key to use?
                                if 'rna' in rgt:
                                    aliquot_rgt_name = '%s %s' % (guide_name, rgt)
                                else:
                                    aliquot_rgt_name = rgt

                                # how much volume is needed for this reagent?
                                transfer_needed = aliquot[rgt + '_ul']

                                # if the transfer is > 0uL
    #                                     if transfer_needed > 0:

                                # step 1: if no aliquots are present for this reagent, create a list of them
                                if aliquot_rgt_name not in self.src_aliquots:
                                    self.src_aliquots[aliquot_rgt_name] = []


                                # step 2: if that list is empty, initialize the aliquot 
                                if len(self.src_aliquots[aliquot_rgt_name]) == 0 :

                                    new_src_aliquot = {
                                        'reagent' : aliquot_rgt_name,
                                        'well_str' : self.src_plating_index_to_str(self.plate_row_index),
                                        'aliquot_index' : -1, 
                                        'active_volume_ul' : 0,
                                        'physical_volume_ul' : 0
                                    }                    

                                    self.plate_row_index += 1 
                                    self.src_aliquots[aliquot_rgt_name].append(new_src_aliquot)                                                    

                                # step 3: check if the transfer requires more volume than is accessible in this aliquot

                                current_aliquot_ptr = self.src_aliquots[aliquot_rgt_name][-1] # get the last aliquot in the list                         
                                aliquot_index = self.src_aliquots[aliquot_rgt_name].index(current_aliquot_ptr)

                                if (transfer_needed + current_aliquot_ptr['active_volume_ul']) > self.effective_transfer_volume:

                                    # no, we don't have enough space left, need to create a new aliquot

                                    # if the transfer would overfill the source, create a new one and put it there instead
                                    new_src_aliquot = {
                                        'reagent' : aliquot_rgt_name,
                                        'aliquot_index' : aliquot_index,
                                        'well_str' : self.src_plating_index_to_str(self.plate_row_index),
                                        'active_volume_ul' : transfer_needed,
                                        'physical_volume_ul' : 0
                                    }  

                                    self.plate_row_index += 1                                    
                                    self.src_aliquots[aliquot_rgt_name].append(new_src_aliquot)

                                    current_aliquot_ptr = self.src_aliquots[aliquot_rgt_name][-1] # get the last aliquot in the list                         
                                    aliquot_index = self.src_aliquots[aliquot_rgt_name].index(current_aliquot_ptr)

                                    aliquot[rgt + '_source'] = current_aliquot_ptr['well_str']
                                    aliquot[rgt + '_source_name'] = '%s %d %s' % (aliquot_rgt_name, current_aliquot_ptr['aliquot_index'], current_aliquot_ptr['well_str'])

                                else:

                                    # yes we have enough, just add 
                                    current_aliquot_ptr['active_volume_ul'] += transfer_needed
                                    current_aliquot_ptr['aliquot_index'] = aliquot_index

                                    aliquot[rgt + '_source'] = current_aliquot_ptr['well_str']
                                    aliquot[rgt + '_source_name'] = '%s %d %s' % (aliquot_rgt_name, current_aliquot_ptr['aliquot_index'], current_aliquot_ptr['well_str'])

    #                                         print("Using existing  aliquot", current_aliquot_ptr)
    #                                         print(aliquot)
    #                                         print('------------------------------------------------------\n')

                            self.aliquots.append(aliquot)

    

In [4]:

# added 21 ul h20
# added 7 ul of 225 nM reporter = 56.25 nM

echo = EchoManager(src_layout=src_layout, 
                src_transfer_range=transfer_range,
                dest_layout=dest_layout,
                reaction_volume=reaction_volume,
                master_mix_ratio=master_mix_ratio,
                starting_concentrations=experiment_starting_conc
               )

echo.add_rna_targets(experiment_rna_guides)

echo.create_aliquots_for_crispr_txtl(experiment_rna_guides, experiment_reporter_dna_concentrations, experiment_guideRNA_dna_concentration, experiment_sgrna_concentrations)

# echo.create_aliquots_for_crispr_txtl(['no reporter control'],  [0], experiment_guideRNA_dna_concentration)

echo.create_aliquots_for_crispr_txtl(['no guide control'], experiment_reporter_dna_concentrations, [0], [0])

echo.create_aliquots_for_crispr_txtl(['blank control'], [0], [0], [0])

echo.generate_source_plating().sort_values(by='well')

,active_volume_ul,reagent,volume,volume_plus,volume_ul,well
9,48.000,master,4.800000e-05,0.000063,63.000,A01
4,17.500,chi,1.750000e-05,0.000033,32.500,A02
6,26.875,h20,2.687500e-05,0.000042,41.875,A03
0,2.100,nuclease,2.100000e-06,0.000017,17.100,A04
2,0.625,g1 rna,6.250000e-07,0.000016,15.625,A05
16,1.050,g1 sgrna,1.050000e-06,0.000016,16.050,A06
3,32.500,reporter,3.250000e-05,0.000048,47.500,A07
15,0.625,g2 rna,6.250000e-07,0.000016,15.625,A08
13,1.050,g2 sgrna,1.050000e-06,0.000016,16.050,A09
10,48.000,master,4.800000e-05,0.000063,63.000,A10


In [5]:
echo.generate_source_plating().groupby('reagent')['volume_ul'].sum()

reagent
blank control rna          15.000
blank control sgrna        15.000
chi                        32.500
g1 rna                     15.625
g1 sgrna                   16.050
g2 rna                     15.625
g2 sgrna                   16.050
h20                        41.875
master                    171.000
no guide control rna       15.000
no guide control sgrna     15.000
nt rna                     15.625
nt sgrna                   16.050
nuclease                   17.100
reporter                   47.500
Name: volume_ul, dtype: float64

In [6]:
# A13 -> A22
# A19 -> A23
# A20 -> A24

In [7]:
echo.generate_transfers()

In [8]:
echo.transfer_df[['name', 'index_str']].head()

,name,index_str
0,g1 - 2.00nM-rep x 0.00nM guide plasmid x 500.0...,A01
1,g1 - 2.00nM-rep x 0.00nM guide plasmid x 500.0...,A02
2,g1 - 2.00nM-rep x 0.00nM guide plasmid x 500.0...,A03
3,g1 - 2.00nM-rep x 0.00nM guide plasmid x 1000....,A04
4,g1 - 2.00nM-rep x 0.00nM guide plasmid x 1000....,A05


In [9]:
prechi = echo.generate_pre_chi_transfer()
postchi = echo.generate_post_chi_transfer()

echo.final_source_df.to_csv('./echo_planning/%s-source-plating.csv' % experiment_name, index=False)
prechi.to_csv('./echo_planning/%s-prechi.csv' % experiment_name, index=False)
postchi.to_csv('./echo_planning/%s-postchi.csv' % experiment_name, index=False)

final_samples = pd.DataFrame(echo.aliquots)
final_samples.to_csv('./echo_planning/%s-sample-list.csv' % experiment_name, index=False)

In [10]:
# postchi.tail()

In [11]:
for n,f in final_samples.iterrows():
    print('%d\t%s \t %s' % (n, f['index_str'], f['name']))

0	A01 	 g1 - 2.00nM-rep x 0.00nM guide plasmid x 500.00nM guide sgRNA  (0)
1	A02 	 g1 - 2.00nM-rep x 0.00nM guide plasmid x 500.00nM guide sgRNA  (1)
2	A03 	 g1 - 2.00nM-rep x 0.00nM guide plasmid x 500.00nM guide sgRNA  (2)
3	A04 	 g1 - 2.00nM-rep x 0.00nM guide plasmid x 1000.00nM guide sgRNA  (0)
4	A05 	 g1 - 2.00nM-rep x 0.00nM guide plasmid x 1000.00nM guide sgRNA  (1)
5	A06 	 g1 - 2.00nM-rep x 0.00nM guide plasmid x 1000.00nM guide sgRNA  (2)
6	A07 	 g1 - 2.00nM-rep x 0.00nM guide plasmid x 2000.00nM guide sgRNA  (0)
7	A08 	 g1 - 2.00nM-rep x 0.00nM guide plasmid x 2000.00nM guide sgRNA  (1)
8	A09 	 g1 - 2.00nM-rep x 0.00nM guide plasmid x 2000.00nM guide sgRNA  (2)
9	A10 	 g1 - 2.00nM-rep x 1.00nM guide plasmid x 0.00nM guide sgRNA  (0)
10	A11 	 g1 - 2.00nM-rep x 1.00nM guide plasmid x 0.00nM guide sgRNA  (1)
11	A12 	 g1 - 2.00nM-rep x 1.00nM guide plasmid x 0.00nM guide sgRNA  (2)
12	B01 	 g2 - 2.00nM-rep x 0.00nM guide plasmid x 500.00nM guide sgRNA  (0)
13	B02 	 g2 - 2.00nM-r

In [12]:
echo.dst_plate_index

42